In [8]:
!pip install pandas scikit-learn

In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample user-item rating data
data = {
    'User ': ['Alice', 'Alice', 'Alice', 'Bob', 'Bob', 'Bob', 'Charlie', 'Charlie', 'Charlie'],
    'Movie': ['Inception', 'Titanic', 'Avatar', 'Inception', 'Avatar', 'The Dark Knight', 'Titanic', 'Avatar', 'The Dark Knight'],
    'Rating': [5, 3, 4, 4, 5, 2, 5, 4, 3]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame to create a user-item matrix
user_item_matrix = df.pivot_table(index='User ', columns='Movie', values='Rating').fillna(0)

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Function to get movie recommendations for a user
def get_movie_recommendations(user, num_recommendations=3):
    # Get the user's ratings
    user_ratings = user_item_matrix.loc[user]

    # Get the similarity scores for the user
    similar_users = user_similarity_df[user].sort_values(ascending=False)

    # Create a Series to hold the weighted ratings
    weighted_ratings = pd.Series()

    for similar_user, similarity_score in similar_users.items():
        if similar_user == user:
            continue  # Skip the user themselves
        # Get the ratings of the similar user
        similar_user_ratings = user_item_matrix.loc[similar_user]
        # Multiply the ratings by the similarity score
        weighted_ratings = weighted_ratings.add(similar_user_ratings * similarity_score, fill_value=0)

    # Normalize the weighted ratings by the sum of similarities
    recommendations = weighted_ratings / similar_users.sum()

    # Filter out already rated movies
    recommendations = recommendations[user_ratings[user_ratings > 0].index]

    # Sort the recommendations
    recommendations = recommendations.sort_values(ascending=False)

    return recommendations.head(num_recommendations)

# Example usage
if __name__ == "__main__":
    user = 'Alice'
    recommendations = get_movie_recommendations(user)
    print(f"Movie recommendations for {user}:")
    print(recommendations)

Movie recommendations for Alice:
Movie
Avatar       2.718484
Inception    1.369355
Titanic      1.258488
dtype: object
